## Machine Learning: Initialize Trained Network

In [1]:
#PyTorchの初期化
import torch
import torchvision

#GPIO
import Jetson.GPIO as GPIO

#ランダム関数
import random

model = torchvision.models.alexnet(pretrained=False)
#種類を増やした場合、変更するところ
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

#モデルの読み込み
model.load_state_dict(torch.load('best_model.pth'))

#カメラのフォーマットを学習済みモデルのフォーマットに合わせ変換
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

#カメラ画像の取得
from jetcam.csi_camera import CSICamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224)
image = camera.read()
image_widget = ipywidgets.Image(format='jpeg')
image_widget.value = bgr8_to_jpeg(image)

#カメラ画像の更新
camera.running = True

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')

#GPU
device = torch.device('cuda')
model = model.to(device)


## Machine Learning: Deduction using the trained network

In [2]:
#推論
import torch.nn.functional as F
import time
import sys
a=0
one_blocked=0.0
two_blocked=0.0
three_blocked=0.0
def update(change):
    global blocked_slider, robot,a
    global one_blocked, two_blocked, three_blocked
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    one_blocked = float(y.flatten()[0])
    two_blocked = float(y.flatten()[1])
    three_blocked = float(y.flatten()[2])
    if(one_blocked > two_blocked and one_blocked > three_blocked):
        a = 0
    elif(two_blocked > one_blocked and two_blocked > three_blocked):
        a = 1
    elif(three_blocked > one_blocked and three_blocked > one_blocked):
        a = 2
    #0　グー rock 
    #1　パー paper
    #2　チョキ scissors   
    
    time.sleep(0.001)
        
update({'new': image})  # we call the function once to intialize


## Gameplay: import libaries

In [3]:
### import randoms
import math

### import colors
from colorama import Fore, Back, Style


## Gameplay: imports GPIO library; actual gameplay process

In [9]:
### GPIO imports
# GPIO is mainly used for controlling lights
# and recieving signals from a button
#
# > Note: GPIO needs to be set every 
#   time when the code bloack is run.
###
import Jetson.GPIO as GPIO

# initialize
GPIO.setwarnings(False)
GPIO.setmode( GPIO.BCM )

# pins initialize
VCC_PIN = 6
GREEN_PIN = 26
YELLOW_PIN = 19
RED_PIN = 13
LOW = GPIO.LOW#False
HIGH = GPIO.HIGH#True

GPIO.setup( [VCC_PIN, GREEN_PIN, YELLOW_PIN, RED_PIN], GPIO.OUT )
GPIO.output( GREEN_PIN, HIGH )
GPIO.output( YELLOW_PIN, HIGH )
GPIO.output( RED_PIN, HIGH )
GPIO.output( VCC_PIN, LOW )

### Functions

def name_of_value(val):
    if val == 0:
        GPIO.output( GREEN_PIN, LOW )
        GPIO.output( YELLOW_PIN, HIGH )
        GPIO.output( RED_PIN, HIGH )
        return "Rock    ";
    if val == 1:
        GPIO.output( GREEN_PIN, HIGH )
        GPIO.output( YELLOW_PIN, HIGH )
        GPIO.output( RED_PIN, LOW )
        return "Paper   ";
    if val == 2:
        GPIO.output( GREEN_PIN, HIGH )
        GPIO.output( YELLOW_PIN, LOW )
        GPIO.output( RED_PIN, HIGH )
        return "Scissor ";
    
### MAIN

# activate hand gesture recognition
display(image_widget)
camera.observe(update, names='value')

# main process
TIME_DELTA = 1
try:
    while True: # forever loop
        # Wait a little and detect hand gesture
        time.sleep( TIME_DELTA )
        sint = a;
        sys.stdout.write("\rYou: %s \tR:%f, P:%f, S:%f" % (name_of_value(sint), one_blocked, two_blocked, three_blocked))
        sys.stdout.flush()
        
except KeyboardInterrupt:
    GPIO.output( VCC_PIN, HIGH )
    GPIO.output( GREEN_PIN, HOGH )
    GPIO.output( YELLOW_PIN, HIGH )
    GPIO.output( RED_PIN, HIGH )
    GPIO.cleanup()
    sys.stdout.write(Style.RESET_ALL + "\n")
    sys.exit(0)
   

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

You: Paper    	R:0.221821, P:0.644641, S:0.133538

NameError: name 'HOGH' is not defined